In [ ]:
import logging
from importlib import import_module

import deep_image_matching as dim
import yaml

logger = dim.setup_logger("dim")

cli_params = {
    "dir": "../assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "strategy": "matching_lowres",
    "quality": "high",
    "tiling": "preselection",
    "skip_reconstruction": False,
    "force": True,
    "camera_options": "../config/cameras.yaml",
    "openmvg": None,
}

In [ ]:
# Build configuration
config = dim.Config(cli_params)
imgs_dir = config.general["image_dir"]
output_dir = config.general["output_dir"]

In [ ]:
# - General configuration
config.general["min_inliers_per_pair"] = 10
config.general["min_inlier_ratio_per_pair"] = 0.2

# - SuperPoint configuration
config.extractor["max_keypoints"] = 8000

# - LightGue configuration
config.matcher["filter_threshold"] = 0.1

# Save configuration to a json file in the output directory
config.save()

In [ ]:
# Initialize ImageMatcher class
matcher = dim.ImageMatcher(config)

# Run image matching
feature_path, match_path = matcher.run()

In [ ]:
# Export in colmap format
with open(config.general["camera_options"], "r") as file:
    camera_options = yaml.safe_load(file)
database_path = output_dir / "database.db"
dim.io.export_to_colmap(
    img_dir=imgs_dir,
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_options=camera_options,
)

In [ ]:
from deep_image_matching import graph

graph.view_graph(database_path, output_dir, imgs_dir)

In [ ]:
reconst_opts = {}
refine_intrinsics = False

# Run reconstruction
model = dim.reconstruction.pycolmap_reconstruction(
    database_path=output_dir / "database.db",
    sfm_dir=output_dir,
    image_dir=imgs_dir,
    options=reconst_opts,
    verbose=config.general["verbose"],
    refine_intrinsics=refine_intrinsics,
)